# Data Validation for LendingClub Loan Data Using Python and TensorFlow Data Validation
### David Lowe
### August 9, 2021

SUMMARY: The project aims to construct a data validation flow using TensorFlow Data Validation (TFDV) and document the end-to-end steps using a template. The Kaggle LendingClub Loan Data dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

INTRODUCTION: The Kaggle dataset owner derived this dataset from the publicly available data of LendingClub.com. Lending Club connects people who need money (borrowers) with people who have money (investors). An investor naturally would want to invest in people who showed a profile of having a high probability of paying back the loan. The dataset uses the lending data from 2007 to 2010, and we will try to predict whether the borrower paid back their loan in full.

Additional Notes: I adapted this workflow from the TensorFlow Data Validation tutorial on TensorFlow.org (https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic). The plan is to build a robust TFDV script for validating datasets in building machine learning models.

CONCLUSION: In this iteration, the data validation workflow helped to validate the features and structures of the training, validation, and test datasets. The workflow also generated statistics over different slices of data which can help track model and anomaly metrics.

Dataset Used: Kaggle LendingClub Loan Data

Dataset ML Model: Binary classification with numerical and categorical attributes

Dataset Reference: https://www.kaggle.com/itssuru/loan-data

Data validation for a machine learning project generally can be broken down into the following tasks:

1. Prepare Environment
2. Generate and Visualize Training Data Statistics
3. Check Anomalies in Validation Dataset
4. Check Anomalies in Test Dataset
5. Check for Data Drift and Skew
6. Display Stats for Data Slices
7. Finalize the Schema

## Task 1 - Prepare Environment

### 1.a) Load libraries and modules

In [1]:
# Set the random seed number for reproducible results
RNG_SEED = 8

In [2]:
# Import packages
import os
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tempfile, urllib, zipfile
import tensorflow_data_validation as tfdv
from tensorflow.python.lib.io import file_io
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics
from tensorflow_metadata.proto.v0 import schema_pb2

### 1.b) Set up the controlling parameters and functions

In [3]:
# Begin the timer for the script processing
start_time_script = datetime.now()

# Set the percentage sizes for splitting the dataset
VAL_SET_RATIO = 0.2
TEST_SET_RATIO = 0.5

# Set TF's logger to only display errors to avoid internal warnings being shown
tf.get_logger().setLevel('ERROR')
print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 1.0.0


### 1.c) Load dataset

In [4]:
# Read CSV data into a dataframe and mark the missing data as NaN
dataset_path = 'https://dainesanalytics.com/datasets/kaggle-lendingclub-loan-data/loan_data.csv'
df_dataset_import = pd.read_csv(dataset_path, header=0)

# Take a peek at the dataframe after import
print(df_dataset_import.head())

   credit.policy             purpose  int.rate  installment  log.annual.inc  \
0              1  debt_consolidation    0.1189       829.10       11.350407   
1              1         credit_card    0.1071       228.22       11.082143   
2              1  debt_consolidation    0.1357       366.86       10.373491   
3              1  debt_consolidation    0.1008       162.34       11.350407   
4              1         credit_card    0.1426       102.92       11.299732   

     dti  fico  days.with.cr.line  revol.bal  revol.util  inq.last.6mths  \
0  19.48   737        5639.958333      28854        52.1               0   
1  14.29   707        2760.000000      33623        76.7               0   
2  11.63   682        4710.000000       3511        25.6               1   
3   8.10   712        2699.958333      33667        73.2               1   
4  14.97   667        4066.000000       4740        39.5               0   

   delinq.2yrs  pub.rec  not.fully.paid  
0            0        0   

In [5]:
df_dataset_import.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9578 non-null   float64
 5   dti                9578 non-null   float64
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9578 non-null   float64
 8   revol.bal          9578 non-null   int64  
 9   revol.util         9578 non-null   float64
 10  inq.last.6mths     9578 non-null   int64  
 11  delinq.2yrs        9578 non-null   int64  
 12  pub.rec            9578 non-null   int64  
 13  not.fully.paid     9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


### 1.d) Splitting Data into Sets

In [6]:
# Standardize the class column to the name of target
df_dataset_import = df_dataset_import.rename(columns={'not.fully.paid':'target'})

In [7]:
# Split the data further into training, validation, and test datasets
df_train, df_val_test = train_test_split(df_dataset_import, test_size=VAL_SET_RATIO, random_state=RNG_SEED)
df_val, df_test = train_test_split(df_val_test, test_size=TEST_SET_RATIO, random_state=RNG_SEED)

# Test data emulates the data that would be submitted for predictions, so it should not have the label column.
df_test = df_test.drop(['target'], axis=1)

print("Training dataset has {} records and {} columns.".format(df_train.shape[0], df_train.shape[1]))
print("Validation dataset has {} records and {} columns.".format(df_val.shape[0], df_val.shape[1]))
print("Test dataset has {} records and {} columns".format(df_test.shape[0], df_test.shape[1]))

Training dataset has 7662 records and 14 columns.
Validation dataset has 958 records and 14 columns.
Test dataset has 958 records and 13 columns


## Task 2 - Generate and Visualize Training Data Statistics

### 2.a) Removing Irrelevant Features

In [8]:
# Define non-essential features to remove
features_to_remove = {}

# Collect features to whitelist while computing the statistics
features_to_keep = [col for col in df_dataset_import.columns if (col not in features_to_remove)]

# Instantiate a StatsOptions class and define the feature_whitelist property
options_train = tfdv.StatsOptions(feature_allowlist=features_to_keep)

# Review the features to generate the statistics
print(options_train.feature_allowlist)

['credit.policy', 'purpose', 'int.rate', 'installment', 'log.annual.inc', 'dti', 'fico', 'days.with.cr.line', 'revol.bal', 'revol.util', 'inq.last.6mths', 'delinq.2yrs', 'pub.rec', 'target']


### 2.b) Generate Statistics for Training Data

In [9]:
train_stats = tfdv.generate_statistics_from_dataframe(df_train, stats_options=options_train)

# get the number of features used to compute statistics
print(f"Number of features used: {len(train_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {train_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {train_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {train_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 14
Number of examples used: 7662
First feature: credit.policy
Last feature: target


### 2.c) Visualize Training Statistics

In [10]:
tfdv.visualize_statistics(train_stats)

### 2.d) Infer Training Schema

In [11]:
# Infer the data schema by using the training statistics previously generated
dataset_schema = tfdv.infer_schema(train_stats)

# Display the data schema
tfdv.display_schema(dataset_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'credit.policy',INT,required,,-
'purpose',STRING,required,,'purpose'
'int.rate',FLOAT,required,,-
'installment',FLOAT,required,,-
'log.annual.inc',FLOAT,required,,-
'dti',FLOAT,required,,-
'fico',INT,required,,-
'days.with.cr.line',FLOAT,required,,-
'revol.bal',INT,required,,-


/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'purpose',"'all_other', 'credit_card', 'debt_consolidation', 'educational', 'home_improvement', 'major_purchase', 'small_business'"


## Task 3 - Check Anomalies in Validation Dataset

### 3.a) Generate Statistics for Validation Data

In [12]:
val_stats = tfdv.generate_statistics_from_dataframe(df_val, stats_options=options_train)

# get the number of features used to compute statistics
print(f"Number of features used: {len(val_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {val_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {val_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {val_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 14
Number of examples used: 958
First feature: credit.policy
Last feature: target


### 3.b) Compare Validation with Training Statistics

In [13]:
tfdv.visualize_statistics(lhs_statistics=val_stats, rhs_statistics=train_stats,
                          lhs_name='VAL_DATASET', rhs_name='TRAIN_DATASET')

### 3.c) Detect Anomalies

In [14]:
val_anomalies = tfdv.validate_statistics(statistics=val_stats, schema=dataset_schema)
tfdv.display_anomalies(val_anomalies)

/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


### 3.d) Fix Validation Data Anomalies in Schema

In [15]:
# Not applicable in this iteration of the data validation exercise

## Task 4 - Check Anomalies in Test Dataset

### 4.a) Generate Statistics for Test Data

In [16]:
# Define a new statistics options by the tfdv.StatsOptions class for the serving data by passing the previously inferred schema
options_test = tfdv.StatsOptions(schema=dataset_schema, infer_type_from_schema=True, feature_allowlist=features_to_keep)

In [17]:
# Generate serving dataset statistics
test_stats = tfdv.generate_statistics_from_dataframe(df_test, stats_options=options_test)

# get the number of features used to compute statistics
print(f"Number of features used: {len(test_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {test_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {test_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {test_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 13
Number of examples used: 958
First feature: credit.policy
Last feature: pub.rec


### 4.b) Compare Test with Training Statistics

In [18]:
test_anomalies = tfdv.validate_statistics(statistics=test_stats, schema=dataset_schema)
tfdv.display_anomalies(test_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'target',Column dropped,Column is completely missing


### 4.c) Fix Test Data Anomalies in Schema

In [19]:
# Not applicable in this iteration of the data validation exercise

### 4.d) Organize Schema based on Environment

In [20]:
# All features are by default in both TRAINING and SERVING environments.
dataset_schema.default_environment.append('TRAINING')
dataset_schema.default_environment.append('TESTING')

In [21]:
# Specify that 'target' feature is not in TEST environment.
tfdv.get_feature(dataset_schema, 'target').not_in_environment.append('TESTING')

# Re-calculate and re-display anomalies with the new environment parameters
test_anomalies = tfdv.validate_statistics(statistics=test_stats, schema=dataset_schema, environment='TESTING')
tfdv.display_anomalies(test_anomalies)

/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Task 5 - Check for Data Drift and Skew

In [22]:
# Not applicable in this iteration of the data validation exercise

## Task 6 - Display Stats for Data Slices

In [23]:
def split_datasets(dataset_list):
    '''
    split datasets.

            Parameters:
                    dataset_list: List of datasets to split

            Returns:
                    datasets: sliced data
    '''
    datasets = []
    for dataset in dataset_list.datasets:
        proto_list = DatasetFeatureStatisticsList()
        proto_list.datasets.extend([dataset])
        datasets.append(proto_list)
    return datasets

In [24]:
def display_stats_at_index(index, datasets):
    '''
    display statistics at the specified data index

            Parameters:
                    index : index to show the anomalies
                    datasets: split data

            Returns:
                    display of generated sliced data statistics at the specified index
    '''
    if index < len(datasets):
        print(datasets[index].datasets[0].name)
        tfdv.visualize_statistics(datasets[index])

In [25]:
def sliced_stats_for_slice_fn(slice_fn, approved_cols, dataframe, schema):
    '''
    generate statistics for the sliced data.

            Parameters:
                    slice_fn : slicing definition
                    approved_cols: list of features to pass to the statistics options
                    dataframe: pandas dataframe to slice
                    schema: the schema

            Returns:
                    slice_info_datasets: statistics for the sliced dataset
    '''
    # Set the StatsOptions
    slice_stats_options = tfdv.StatsOptions(schema=schema,
                                            slice_functions=[slice_fn],
                                            infer_type_from_schema=True,
                                            feature_allowlist=approved_cols)
    
    # Convert Dataframe to CSV since `slice_functions` works only with `tfdv.generate_statistics_from_csv`
    CSV_PATH = 'slice_sample.csv'
    dataframe.to_csv(CSV_PATH)
    
    # Calculate statistics for the sliced dataset
    sliced_stats = tfdv.generate_statistics_from_csv(CSV_PATH, stats_options=slice_stats_options)
    
    # Split the dataset using the previously defined split_datasets function
    slice_info_datasets = split_datasets(sliced_stats)
    
    return slice_info_datasets

In [26]:
# Generate slice function for one of the features
slice_fn = slicing_util.get_feature_value_slicer(features={'purpose': None})

# Generate stats for the sliced dataset
slice_datasets = sliced_stats_for_slice_fn(slice_fn, features_to_keep, dataframe=df_train, schema=dataset_schema)

# Print name of slices for reference
print(f'Statistics generated for:\n')
print('\n'.join([sliced.datasets[0].name for sliced in slice_datasets]))

# Display at index 1 as an example
display_stats_at_index(1, slice_datasets) 

Statistics generated for:

All Examples
purpose_debt_consolidation
purpose_credit_card
purpose_home_improvement
purpose_educational
purpose_all_other
purpose_small_business
purpose_major_purchase
purpose_debt_consolidation


## Task 7 - Finalize the Schema

In [27]:
# Create output directory
OUTPUT_DIR = "output"
file_io.recursive_create_dir(OUTPUT_DIR)

# Use TensorFlow text output format pbtxt to store the schema
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')

# write_schema_text function expect the defined schema and output path as parameters
tfdv.write_schema_text(dataset_schema, schema_file) 

In [28]:
print ('Total time for the script:',(datetime.now() - start_time_script))

Total time for the script: 0:00:08.307018
